In [1]:
import pandas as pd
import re

In [6]:
# Load the CSV and Excel files
cve_df = pd.read_csv('../data/cve_mapping.csv')
cwe_df = pd.read_excel('../data/cve_to_cwe.xlsx')

# Clean up and drop unnecessary columns in cve_df
cve_df = cve_df.drop(columns=[
    'attack_version', 'technology_domain', 'references', 'comments', 
    'organization', 'creation_date', 'last_update', 
    'mapping_framework_version', 'mapping_framework', 'Unnamed: 0'])

# Rename columns in cwe_df and drop description
cwe_df.rename(columns={
    "CVE-ID": "capability_id", "CVSS-V3": "cvss_v3", "CVSS-V2": "cvss_v2", 
    "SEVERITY": "severity", "CWE-ID": "cwe_id", "ID": "id"
}, inplace=True)
cwe_df = cwe_df.drop(columns=['DESCRIPTION'])
cwe_df['severity'] = cwe_df['severity'].str.lower()

# Merge cve_df with cwe_df on 'capability_id'
cve_df = pd.merge(cve_df, cwe_df, how='left', on='capability_id')

# Extract the year from the 'capability_group' and rename columns
cve_df['capability_group'] = cve_df['capability_group'].str.extract(r'(\d{4})')
cve_df.rename(columns={'capability_group': 'year', 'capability_id': 'cve'}, inplace=True)

# Use CVSS v3 if available, otherwise fallback to CVSS v2
cve_df['cvss'] = cve_df['cvss_v3'].combine_first(cve_df['cvss_v2'])

# Drop unneeded columns
cve_df.drop(columns=['id', 'cvss_v3', 'cvss_v2'], inplace=True)

# Sort by 'year' to get the latest entries first
df_sorted = cve_df.sort_values('year', ascending=False)

# Strip any leading/trailing spaces in column names
df_sorted.columns = df_sorted.columns.str.strip()

# Grouping and processing the data
result = df_sorted.groupby('attack_object_id').agg({
    'cve': lambda x: ', '.join(x),                        # Join CVEs by comma
    'cvss': ['max', 'mean'],                              # Get highest and average CVSS
    'cwe_id': lambda x: ', '.join(x),                     # Join CWE IDs by comma
    'mapping_type': lambda x: x.value_counts().to_dict()   # Count mapping types
}).reset_index()

# Flatten multi-level column names
result.columns = ['ttp', 'cves', 'high_cvss', 'avg_cvss', 'cwes', 'mapping_type_count']

# result.to_excel('cvecve.xlsx')

# Display the result
display(result)

,ttp,cves,high_cvss,avg_cvss,cwes,mapping_type_count
0,T1480,CVE-2012-4681,10.0,10.000000,NVD-CWE-noinfo,{'uncategorized': 1}
1,T1565.001,"CVE-2020-5539, CVE-2015-7925",8.0,7.250000,"CWE-639, CWE-352",{'uncategorized': 2}
2,T0803,CVE-2018-17924,8.6,8.600000,CWE-306,{'secondary_impact': 1}
3,T0804,CVE-2018-17924,8.6,8.600000,CWE-306,{'secondary_impact': 1}
4,T0812,CVE-2018-14847,9.1,9.100000,CWE-22,{'uncategorized': 1}
...,...,...,...,...,...,...
137,T1575,"CVE-2019-10987, CVE-2019-13541, CVE-2018-17911...",9.8,8.550000,"CWE-787, CWE-787, CWE-787, CWE-119",{'primary_impact': 4}
138,T1588.004,CVE-2019-3762,7.5,7.500000,CWE-295,{'exploitation_technique': 1}
139,T1589,CVE-2020-15132,5.3,5.300000,CWE-209,{'primary_impact': 1}
140,T1600,CVE-2018-11069,5.9,5.900000,CWE-327,{'primary_impact': 1}


In [10]:

def extract_cve_year(cve):
    match = re.search(r'CVE-(\d{4})', cve)
    return int(match.group(1)) if match else None

# Assuming 'result' is a DataFrame with a 'cves' column
df = result

# Extract years directly and filter None values in one step
df['cve_years'] = [[extract_cve_year(cve) for cve in cves.split(', ') if cve] for cves in df['cves']]

# Flatten years with early stopping for None values
all_years = [year for years in df['cve_years'] for year in years if year is not None and years[0] is not None]

# Find latest year dynamically
latest_year = max(all_years) if all_years else None

# Combine logic for finding most recent year
df['most_recent_cve_year'] = df['cve_years'].apply(lambda years: max([y for y in years if y is not None]) if years else None)

# Group by TTP and count
ttp_year_summary = df.groupby('most_recent_cve_year').size().reset_index(name='ttp_count')

# Sort and filter
ttp_year_summary = ttp_year_summary.sort_values(by='most_recent_cve_year', ascending=False)
latest_techniques = df[df['most_recent_cve_year'] == latest_year]

# Results
print(f"Latest CVE Year: {latest_year}")
print(f"TTPs using CVEs from the latest year ({latest_year}):")
display(latest_techniques[['ttp', 'most_recent_cve_year']])

print("\nSummary of TTPs by CVE Year:")
display(ttp_year_summary)


Latest CVE Year: 2020
TTPs using CVEs from the latest year (2020):


,ttp,most_recent_cve_year
1,T1565.001,2020
5,T0814,2020
7,T0826,2020
11,T0842,2020
13,T0859,2020
...,...,...
130,T1566.001,2020
131,T1566.002,2020
133,T1574,2020
139,T1589,2020



Summary of TTPs by CVE Year:


,most_recent_cve_year,ttp_count
10,2020,68
9,2019,24
8,2018,17
7,2017,9
6,2016,5
5,2015,2
4,2014,7
3,2013,2
2,2012,4
1,2011,1


In [14]:
def extract_cve_year(cve):
    match = re.search(r'CVE-(\d{4})', cve)
    return int(match.group(1)) if match else None

# Assuming 'result' is a DataFrame with a 'cves' column
df = result

# Extract years directly and filter None values in one step
df['cve_years'] = [[extract_cve_year(cve) for cve in cves.split(', ') if cve] for cves in df['cves']]

# Flatten years with early stopping for None values

# Find latest year dynamically

# Combine logic for finding most recent year
df['most_recent_cve_year'] = df['cve_years'].apply(lambda years: max([y for y in years if y is not None]) if years else None)

# Group by TTP and count
ttp_year_summary = df.groupby('most_recent_cve_year').size().reset_index(name='ttp_count').sort_values(by='most_recent_cve_year', ascending=False)

# Sort
ttp_year_summary = ttp_year_summary

# Print the result
print("\nSummary of TTPs by CVE Year:")
display(ttp_year_summary)


Summary of TTPs by CVE Year:


,most_recent_cve_year,ttp_count
10,2020,68
9,2019,24
8,2018,17
7,2017,9
6,2016,5
5,2015,2
4,2014,7
3,2013,2
2,2012,4
1,2011,1
